# lorapy symbol aggregation

In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import os, sys
from loguru import logger
logger.remove(None)
logger.add(sys.stdout, colorize=True)

import numpy as np
import pandas as pd
from six.moves import cPickle
import tqdm
from tqdm import tqdm_notebook
import warnings 
import scipy.signal as spsig
import multiprocessing
from functools import partial 

import matplotlib.pyplot as plt 
# plt.style.use('dark_background')
%matplotlib inline

In [2]:
import lorapy

## setup 

In [3]:
_BASE_DATA_DIR = pathlib.Path('../data')

BENCHTOP_DATA_DIR = _BASE_DATA_DIR.joinpath('lora-benchtop-recording')
DOTP_DATA_DIR = _BASE_DATA_DIR.joinpath('symbol-ref')
AGG_SYMBOL_DIR = _BASE_DATA_DIR.joinpath('aggregated-symbols')
PROC_SYMBOL_DIR = _BASE_DATA_DIR.joinpath('processed-symbols')
FILT_SYMBOL_DIR = _BASE_DATA_DIR.joinpath('filtered-symbols')

# load

In [4]:
import random
import warnings 

rand = lambda mx: random.randint(0, mx - 1)

def _load_matching_dotp(bw: int, sf: int):
    return ploader.filter(bw=bw, sf=sf)[0]


def _plot_check(filtered_syms, num=20):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        fig, ax = plt.subplots(num, figsize=(14, 10))

        for idx in range(num):
            ax[idx].plot(filtered_syms[rand(filtered_syms.shape[0])])
            
            
def _format_output_path(out_dir, params): 
    bw, sf, att = params
    filename = f'lora_BW{bw}_SF{sf}_915MHz_20ksps_Att{att}_combined.p'
    
    out_path = out_dir.joinpath(filename)
    return out_path


def _save_symbols(data, out_dir, params):
    out_path = _format_output_path(out_dir, params)
    
    with out_path.open('wb') as outfile:
        cPickle.dump(data, outfile)

def _combine_all_syms(file_list, params):
    combined_syms = np.vstack([
        file.load().to_signal().data
        for file in file_list
    ])
    
    logger.warning(f'[{params}]  combined {combined_syms.shape[0]} total symbols [{combined_syms.shape}]')
    return combined_syms

def combine_and_save(file_list, params, out_dir):
    combined_syms = _combine_all_syms(file_list, params)
    if combined_syms.shape[0] == 0:
        logger.info('no syms for params')
        return 
    
    _save_symbols(combined_syms, out_dir, params)

## files

In [5]:
symloader = lorapy.load_dotp(FILT_SYMBOL_DIR, glob_pattern='**/*.p')
symloader.file_list

2020-04-11 17:29:20.065 | DEBUG    | lorapy.io._base_loader:_validate_data_path:140 - set datafile directory: ../data/filtered-symbols
2020-04-11 17:29:20.073 | INFO     | lorapy.io._base_loader:_process_data_dir:153 - found 150 data file(s)


[DotPFile(id=0 | name='lora_BW1_SF10_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=1 | name='lora_BW2_SF12_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=2 | name='lora_BW1_SF10_915MHz_20ksps_Att140_v0.p'),
 DotPFile(id=3 | name='lora_BW1_SF12_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=4 | name='lora_BW1_SF12_915MHz_20ksps_Att60_v0.p'),
 DotPFile(id=5 | name='lora_BW7_SF10_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=6 | name='lora_BW2_SF10_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=7 | name='lora_BW9_SF12_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=8 | name='lora_BW2_SF12_915MHz_20ksps_Att60_v0.p'),
 DotPFile(id=9 | name='lora_BW2_SF10_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=10 | name='lora_BW9_SF11_915MHz_20ksps_Att100_v0.p'),
 DotPFile(id=11 | name='lora_BW7_SF10_915MHz_20ksps_Att100_v0.p'),
 DotPFile(id=12 | name='lora_BW9_SF11_915MHz_20ksps_Att80_v0.p'),
 DotPFile(id=13 | name='lora_BW1_SF10_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=14 | name='lora_BW7_SF11_915MHz_20ksps_Att80_v0.p'),
 DotPFile(id=15 | nam

# combine and aggregate

In [6]:
import itertools

## setup

In [7]:
loaded_files = [file.load() for file in symloader.filegen]

bw_opts = set(file.bw for file in loaded_files)
sf_opts = set(file.sf for file in loaded_files)
att_opts = set(file.att for file in loaded_files)

all_opts = (bw_opts, sf_opts, att_opts)

2020-04-11 17:29:20.129 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:20.436 | INFO     | lorapy.datafile._base_file:load:61 - loaded 8907872 samples from file
2020-04-11 17:29:20.438 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 51197 and packet length: 1548709
2020-04-11 17:29:20.741 | INFO     | lorapy.datafile._base_file:load:61 - loaded 9573839 samples from file
2020-04-11 17:29:20.743 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:20.877 | INFO     | lorapy.datafile._base_file:load:61 - loaded 8178348 samples from file
2020-04-11 17:29:20.878 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 76801 and packet length: 2323230
2020-04-11 17:29:21.109 | INFO     | lorapy.datafile._base_file:load:61 - loaded 8448110 samples from fil

In [8]:
param_list = sorted(list(
    itertools.product(*all_opts)
))

param_list[:5]

[(1, 10, 0), (1, 10, 20), (1, 10, 40), (1, 10, 60), (1, 10, 80)]

## load combine save

In [9]:
logger.add('dev/symbol-agg-logs.log', level='WARNING')

2

In [10]:
for params in tqdm.tqdm(param_list):
    file_list = symloader.filter(*params)
    if len(file_list) == 0:
        continue
    
    combine_and_save(file_list, params, AGG_SYMBOL_DIR)


  0%|          | 0/120 [00:00<?, ?it/s]

2020-04-11 17:29:48.306 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:48.513 | INFO     | lorapy.datafile._base_file:load:61 - loaded 8907872 samples from file
2020-04-11 17:29:48.516 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:48.847 | INFO     | lorapy.datafile._base_file:load:61 - loaded 12497898 samples from file
2020-04-11 17:29:49.045 | WARNING  | __main__:_combine_all_syms:40 - [(1, 10, 0)]  combined 1115 total symbols [(1115, 19198)]


  1%|          | 1/120 [00:01<02:38,  1.33s/it]

2020-04-11 17:29:49.650 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:49.873 | INFO     | lorapy.datafile._base_file:load:61 - loaded 9963762 samples from file
2020-04-11 17:29:49.875 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 19198 and packet length: 580739
2020-04-11 17:29:50.021 | INFO     | lorapy.datafile._base_file:load:61 - loaded 12497898 samples from file
2020-04-11 17:29:50.165 | WARNING  | __main__:_combine_all_syms:40 - [(1, 10, 20)]  combined 1170 total symbols [(1170, 19198)]


  2%|▏         | 2/120 [00:02<02:29,  1.27s/it]

...

 97%|█████████▋| 116/120 [01:01<00:01,  2.79it/s]

2020-04-11 17:30:50.208 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 1599 and packet length: 48369
2020-04-11 17:30:50.270 | INFO     | lorapy.datafile._base_file:load:61 - loaded 4557150 samples from file
2020-04-11 17:30:50.290 | WARNING  | __main__:_combine_all_syms:40 - [(9, 12, 80)]  combined 2850 total symbols [(2850, 1599)]


 98%|█████████▊| 117/120 [01:02<00:00,  3.17it/s]

2020-04-11 17:30:50.426 | DEBUG    | lorapy.datafile.encoding:compute_params:15 - computed samples per symbol: 1599 and packet length: 48369
2020-04-11 17:30:50.744 | INFO     | lorapy.datafile._base_file:load:61 - loaded 7948629 samples from file
2020-04-11 17:30:50.866 | WARNING  | __main__:_combine_all_syms:40 - [(9, 12, 100)]  combined 4971 total symbols [(4971, 1599)]


100%|██████████| 120/120 [01:03<00:00,  1.90it/s]
